# **Ajuste de Hiperparámetros para el DataSet "Enfermedades Cardíacas de UCI":**

## 1. Carga y exploración del dataset

In [5]:
import pandas as pd

# Cargamos el dataset
df = pd.read_csv("heart.csv")


In [6]:
#  Revisamos las primeras filas del dataset
print("Primeras filas del dataset:")
display(df.head())

Primeras filas del dataset:


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [7]:
# Información general del dataset
print("\nInformación del dataset:")
df.info()


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [8]:
# Revisamos estadísticas descriptivas
print("\nEstadísticas descriptivas:")
display(df.describe())


Estadísticas descriptivas:


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [9]:
# Verificamos si hay valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


Valores nulos por columna:
Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64


## 2. Conjuntos de entrenamiento y prueba

In [10]:
from sklearn.model_selection import train_test_split

# Separamos variables predictoras (X) y variable objetivo (y)
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"]

# Divimos en train y test (80% - 20%), estratificando por la variable objetivo
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Verificamos tamaños
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (734, 11)
Tamaño del conjunto de prueba: (184, 11)


## 3. Preprocesamiento y Pipeline

In [1]:
# Variables categoricas
categorical_features = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]

In [2]:
# Variables numéricas
numerical_features = ["Age", "RestingBP", "Cholesterol", "FastingBS", "MaxHR", "Oldpeak"]

In [11]:
# Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Definimos transformadores
categorical_transformer = OneHotEncoder(drop="first", handle_unknown="ignore")
numerical_transformer = StandardScaler()

# Combinamos transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

# Creamos el pipeline general (por ahora solo el preprocesamiento)
preprocessing_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor)
])

# Ajustamos el pipeline al conjunto de entrenamiento
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

# Visualizamos las dimensiones después de transformar
print("Shape después del preprocesamiento - Train:", X_train_processed.shape)
print("Shape después del preprocesamiento - Test:", X_test_processed.shape)


Shape después del preprocesamiento - Train: (734, 15)
Shape después del preprocesamiento - Test: (184, 15)


## 4. Modelo Baseline (Clasificación) e hiperparámetros por defecto

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Creamos el pipeline completo (preprocesamiento + modelo)
baseline_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Realizamos la validación cruzada usando ROC AUC
baseline_scores = cross_val_score(
    baseline_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring="roc_auc"
)

# Revisamos los resultados
print("ROC AUC (modelo baseline - validación cruzada):")
print("Scores:", baseline_scores)
print("Promedio:", baseline_scores.mean())


ROC AUC (modelo baseline - validación cruzada):
Scores: [0.95938086 0.93106996 0.93275346 0.89973812 0.88803419]
Promedio: 0.9221953184825814


## 5. Definimos el espacio de hiperparámetros para *RandomForestClassifier*

In [13]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

# Definimos el espacio de hiperparámetros
param_distributions = {
    "classifier__n_estimators": randint(100, 1000),
    "classifier__max_depth": randint(5, 50),
    "classifier__min_samples_split": randint(2, 10),
    "classifier__min_samples_leaf": randint(1, 10),
    "classifier__max_features": ["sqrt", "log2", None],
    "classifier__bootstrap": [True, False]
}


## 6. Ajuste de hiperparámetros

## 6.1.  Ajuste de hiperparámetros con búsqueda aleatoria

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Definimos nuevamente el pipeline 
pipeline_random = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Configuramos la búsqueda aleatoria
random_search = RandomizedSearchCV(
    estimator=pipeline_random,
    param_distributions=param_distributions,
    n_iter=50,  # número de combinaciones a probar
    cv=5,
    scoring="roc_auc",
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Ajustamos sobre el conjunto de entrenamiento
random_search.fit(X_train, y_train)

# 4. Ver mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Mejor score de validación cruzada
print("\nMejor score ROC AUC (validación cruzada):", random_search.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores hiperparámetros encontrados:
{'classifier__bootstrap': False, 'classifier__max_depth': 31, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 5, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 918}

Mejor score ROC AUC (validación cruzada): 0.928915940027051


## 6.2. Ajuste de hiperparámetros con Optuna

In [15]:
pip install optuna


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    # Sugerimos hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])

    # Creamos el pipeline con los hiperparámetros sugeridos
    model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bootstrap,
            random_state=42
        ))
    ])

    # Realizamos la validación cruzada (ROC AUC)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc", n_jobs=-1)
    return score.mean()

# Ejecutamos la optimización
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Mostramos los mejores parámetros
print("Mejores hiperparámetros encontrados por Optuna:")
print(study.best_params)

print("\nMejor ROC AUC promedio (validación cruzada):", study.best_value)


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-19 11:44:21,401] A new study created in memory with name: no-name-3e043bde-5dfe-4e9b-943b-2db0e8785ef5
[I 2025-05-19 11:44:28,458] Trial 0 finished with value: 0.9155933316908926 and parameters: {'n_estimators': 654, 'max_depth': 15, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': None, 'bootstrap': True}. Best is trial 0 with value: 0.9155933316908926.
[I 2025-05-19 11:44:31,517] Trial 1 finished with value: 0.8733383238261286 and parameters: {'n_estimators': 522, 'max_depth': 17, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None, 'bootstrap': False}. Best is trial 0 with value: 0.9155933316908926.
[I 2025-05-19 11:44:36,557] Trial 2 finished with value: 0.9152590178

Mejores hiperparámetros encontrados por Optuna:
{'n_estimators': 903, 'max_depth': 32, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': False}

Mejor ROC AUC promedio (validación cruzada): 0.9295925834679222


## 7. Comparación de los modelos con validación cruzada

In [17]:
from sklearn.base import clone

# Modelo baseline
baseline_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])
baseline_scores = cross_val_score(baseline_model, X_train, y_train, cv=5, scoring="roc_auc")

# Mejor modelo de RandomizedSearchCV
best_random_model = random_search.best_estimator_
random_search_scores = cross_val_score(best_random_model, X_train, y_train, cv=5, scoring="roc_auc")

# Mejor modelo de Optuna
best_optuna_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        **study.best_params, random_state=42
    ))
])
optuna_scores = cross_val_score(best_optuna_model, X_train, y_train, cv=5, scoring="roc_auc")

# Mostrar resultados
print("ROC AUC promedio en entrenamiento (validación cruzada):")
print(f"Baseline:       {baseline_scores.mean():.4f}")
print(f"Random Search:  {random_search_scores.mean():.4f}")
print(f"Optuna:         {optuna_scores.mean():.4f}")


ROC AUC promedio en entrenamiento (validación cruzada):
Baseline:       0.9222
Random Search:  0.9289
Optuna:         0.9296


## 8. Evaluación en el conjunto de prueba

In [18]:
from sklearn.metrics import roc_auc_score

# Entrenamos los tres modelos completos sobre todo X_train
baseline_model.fit(X_train, y_train)
best_random_model.fit(X_train, y_train)
best_optuna_model.fit(X_train, y_train)

# Predecimos probabilidades para calcular ROC AUC
baseline_proba = baseline_model.predict_proba(X_test)[:, 1]
random_proba = best_random_model.predict_proba(X_test)[:, 1]
optuna_proba = best_optuna_model.predict_proba(X_test)[:, 1]

# Calculamos ROC AUC en el conjunto de prueba
baseline_auc = roc_auc_score(y_test, baseline_proba)
random_auc = roc_auc_score(y_test, random_proba)
optuna_auc = roc_auc_score(y_test, optuna_proba)

# Verificamos los resultados
print("ROC AUC en conjunto de prueba:")
print(f"Baseline:       {baseline_auc:.4f}")
print(f"Random Search:  {random_auc:.4f}")
print(f"Optuna:         {optuna_auc:.4f}")


ROC AUC en conjunto de prueba:
Baseline:       0.9314
Random Search:  0.9272
Optuna:         0.9289


## 9. Conceptualización y conclusiones

**Conceptualización:**

Este trabajo tuvo como objetivo aplicar técnicas de ajuste de hiperparámetros para mejorar el desempeño de un modelo de clasificación binaria, utilizando el dataset "Heart Disease UCI". Se trabajó bajo el enfoque de aprendizaje supervisado, empleando un modelo de bosque aleatorio (RandomForestClassifier) como clasificador base.
Se seleccionó un dataset cn características numéricas y una variable onjetivo binaria, para después preparar el conjunto de datos, con entrenamiento y prueba, luego implementando un pipeline de procesacimiento para codificar las variables categoricas. Entrenamos un modelo baseline con hiperparámetros por defecto. Realizamos:
- Optimización del modelo mediante una búsqueda aleatoria con *RandomizedSearchCV* y optimización con *Optuna*, usando validación cruzada con ROC y AUC.

Finalmente comparamos el desempeño de los tres modelos (baseline, Random Search y Optuna).


**Conclusiones:**

- El modelo baseline obtuvo un rendimiento más eficiente tanto en entrenamiento como en prueba, con un ROC AUC en test de 0.9314.

- La búsqueda aleatoria y la optimización bayesiana dieron como resultado leves mejorías en el entrenamiento, aunque no superaron significativamente al baseline en el conjunto de prueba.

- Esto sugiere que el modelo con hiperparámetros por defecto ya está bien ajustado al problema, y que las técnicas de ajuste no necesariamente garantizan mejoras sustanciales cuando el modelo base es fuerte.

- La optimización bayesiana (Optuna) resultó ser una herramienta muy útil para mejorar el modelo, ya que busca los mejores hiperparámetros de manera más ordenada y rápida, sin necesidad de probar todas las combinaciones posibles.